本文件仅包含一些测试代码

In [65]:
# test config.json load and parse
import json
from vastai_client import VastClient

with open("config.json") as j:
    cfg = json.load(j)

print(cfg)

flag = False
api_key = cfg['vast_api_key']
eth_address = cfg['eth_address']
machine_configs = cfg['machine_configs']
client = VastClient(api_key)

2023-11-04 12:52:13.343 | INFO     | __main__:<module>:9 - Loaded config: {'url': 'https://console.vast.ai', 'vast_api_key': 'bdc70ca54bf58108c0d3305b5b260e7828003c216511d28f6ee81f7b69b7a18f', 'eth_address': '0xEfE59673DE05Bce7CF3aae176A78D03Be15dA676', 'machine_configs': [{'gpu_name': 'RTX_4090', 'gpu_nums': 2, 'dph': 0.8, 'cuda_compute': 89, 'machine_nums': 1}]}


In [66]:
# search offers
import time
from requests.exceptions import HTTPError
from vastai_client import VastClient

def search_offers(cfg):
    global machine_nums, flag, eth_address
    print(cfg)
    try:
        offers = client.search_offers(
            # reliability > 0.98
            search_query=f'num_gpus = {cfg["gpu_nums"]} dph <= {cfg["dph"]} gpu_name = {cfg["gpu_name"]} '
                        'cuda_vers >= 12.0 verified=false external=false rentable=true',
            sort_order='dph, reliability-'
        )
    except ValueError as e:
        print("Search offers error: ", e)
        # print("sleep 5 seconds")
        # time.sleep(5)
        raise e

    return offers

# test api limit
# limit = 10 # pre 1s request once
# times = 1000 # pre 1000 request limit - 1
# while limit > 0:
#     offers = search_offers(api_key, machine_configs[0])
#     print(offers)
#     print(f"sleep {limit} s, remain {times}")
#     time.sleep(limit)
#     times -= 1
#     if times == 0:
#         times = 1000
#         limit -= 1

In [67]:
# create instances
def create_instance(offer, cfg):
    try:
        res = client.create_instance(id=offer.id, image="nvidia/cuda:12.0.1-devel-ubuntu20.04", ssh=True,
                                 env=f"-e ETH_ADDRESS={eth_address} -e CUDA_COMPUTE={cfg['cuda_compute']} -e GPU_NUMS={cfg['gpu_nums']}",
                                 onstart="onstart.sh")
    except HTTPError as e:
        print("Create instance error:", e) 
        return False
    rj = eval(res)
    if not rj['success']:
        print("Create instance error:", res)
        return False
    return True

In [68]:
# test search and create
def robIt(cfg):
    limit = cfg['machine_nums']
    while True:
        offers = search_offers(cfg)
        if not offers:
            print("not available offers, sleep 5 seconds")
            time.sleep(5)
            continue
        for offer in offers:
            if create_instance(offer, cfg):
                limit -= 1
                print(f"Create instance success {offer.id}")
            else:
                print(f"Create instance failed {offer.id}")
            
            if limit == 0:
                break
    

In [69]:
# test mulit process
from threading import Thread

processes = []
for cfg in machine_configs:
    thread = Thread(target=robIt, args=(cfg,))
    thread.start()
    processes.append(thread)

# wait thread finish
for process in processes:
    process.join()

print("program done!!!")

{'gpu_name': 'RTX_4090', 'gpu_nums': 2, 'dph': 0.8, 'cuda_compute': 89, 'machine_nums': 1}


2023-11-04 12:52:29.139 | ERROR    | __main__:create_instance:8 - Http Error: 429 Client Error: Too Many Requests for url: https://console.vast.ai/api/v0/asks/7309118/?api_key=bdc70ca54bf58108c0d3305b5b260e7828003c216511d28f6ee81f7b69b7a18f
2023-11-04 12:52:29.141 | ERROR    | __main__:robIt:15 - rend 7309118 failed!
2023-11-04 12:52:29.141 | INFO     | __main__:robIt:18 - sleep 10 s
2023-11-04 12:52:40.150 | ERROR    | __main__:create_instance:8 - Http Error: 429 Client Error: Too Many Requests for url: https://console.vast.ai/api/v0/asks/7309110/?api_key=bdc70ca54bf58108c0d3305b5b260e7828003c216511d28f6ee81f7b69b7a18f
2023-11-04 12:52:40.151 | ERROR    | __main__:robIt:15 - rend 7309110 failed!
2023-11-04 12:52:40.152 | INFO     | __main__:robIt:18 - sleep 10 s


KeyboardInterrupt: 

In [75]:
# test : get owner instances and insert to db
import json
from vastai_client import VastClient

with open("config.json") as j:
    cfg = json.load(j)
print(cfg)
flag = False
api_key = cfg['vast_api_key']
eth_address = cfg['eth_address']
machine_configs = cfg['machine_configs']
client = VastClient(api_key)
instances = client.get_instances()
print(instances)

{'url': 'https://console.vast.ai', 'vast_api_key': 'bdc70ca54bf58108c0d3305b5b260e7828003c216511d28f6ee81f7b69b7a18f', 'eth_address': '0xEfE59673DE05Bce7CF3aae176A78D03Be15dA676', 'machine_configs': [{'gpu_name': 'RTX_4090', 'gpu_nums': 2, 'dph': 0.8, 'cuda_compute': 89, 'machine_nums': 1}]}
{'actual_status': 'exited', 'bundle_id': 330242520, 'bw_nvlink': 0.0, 'compute_cap': 890, 'cpu_cores': 192, 'cpu_cores_effective': 27.428544000000002, 'cpu_name': 'AMD EPYC 7R12 48-Core Processor', 'cpu_ram': 515814, 'cpu_util': 7.22823639490306, 'cuda_max_good': 12.0, 'cur_state': 'stopped', 'direct_port_count': 14, 'direct_port_end': -1, 'direct_port_start': 65535, 'disk_bw': 2029.0, 'disk_name': 'nvme', 'disk_space': 10.0, 'disk_util': 0.7200000286102295, 'dlperf': 91.649582, 'dlperf_per_dphtotal': 113.87005874029335, 'dph_base': 0.8, 'dph_total': 0.8048611111111111, 'driver_version': '525.105.17', 'duration': 1661201.6830925941, 'end_date': 1700736535.0, 'external': False, 'flops_per_dphtotal':

OperationalError: no such table: instances

In [64]:
import requests
url = "https://console.vast.ai/api/v0/instances?api_key=a7698ab720b9525b36abe8c247cef30344d26d05cacbac80889a4b8e166fa022"
response = requests.request("GET", url, headers={'Accept': 'application/json'})
print(response.text)

{"instances": [{"is_bid": false, "inet_up_billed": 5888.0, "inet_down_billed": 182272.0, "external": false, "webpage": null, "logo": "/static/logos/vastai_small2.png", "rentable": true, "compute_cap": 890, "credit_balance": null, "credit_discount": null, "credit_discount_max": 0.1, "driver_version": "525.105.17", "cuda_max_good": 12.0, "machine_id": 10298, "hosting_type": null, "public_ipaddr": "114.34.116.46", "geolocation": "Taipei City, TW", "flops_per_dphtotal": 203.23737069887838, "dlperf_per_dphtotal": 138.79521628990508, "reliability2": 0.9971786, "host_run_time": 2592000.0, "client_run_time": 100000000.0, "host_id": 1801, "id": 7416022, "bundle_id": 329689876, "num_gpus": 2, "total_flops": 163.57785600000003, "min_bid": 0.7999444, "dph_base": 0.8, "dph_total": 0.8048611111111111, "gpu_name": "RTX 4090", "gpu_ram": 24564, "gpu_totalram": 49128, "vram_costperhour": 1.6382940708172756e-05, "gpu_display_active": false, "gpu_mem_bw": 1810.4, "bw_nvlink": 0.0, "direct_port_count": 14

> 记录一些原始api的测试情况

## 获取当前用户设备列表
```
url = "https://console.vast.ai/api/v0/instances?api_key=" + api_key

payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
```

## 查询符合要求的机器
```
from urllib.parse import quote
import json

q = {
    "disk_space": {"gte": 16},
    "verified": {"eq": "true"},
    "rentable": {"eq": "true"},
    "num_gpus": {"gte": 2, "lte": 2},
    "gpu_option": {"eq": "RTX 3090"},
    "sort_option": {"0": ["dphtotal", "asc"], "1": ["total_flops", "asc"]},
    "order": [["dphtotal", "asc"]],
    "gpu_name": {"eq": "RTX 3090"},
    "allocated_storage": 16,
    "extra_ids": [],
    "type": "ask"
}
url = "https://console.vast.ai/api/v0/bundles?q=" + quote(json.dumps(q))
payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(json.dumps(json.loads(response.text), indent=4, sort_keys=True))
```

## 创建实例
```
url = "https://console.vast.ai/api/v0/asks/{id}/?api_key=" + api_key

payload = {
    "client_id": "me",
    "image": "nvidia/cuda:12.0.1-devel-ubuntu20.04",
    "runtype": "ssh ssh_direc ssh_proxy",
    "image_login": None,
    "python_utf8": False,
    "lang_utf8": False,
    "use_jupyter_lab": False,
    "jupyter_dir": None,
    "create_from": None,
    "force": False,
    "template_id": 27230
}
headers = {
    'Accept': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)

print(response.text)
```

In [ ]:
# python 通过ssh远程执行命令
# import paramiko
